# Knowledge and Data Mining project report

## Project 2: Learning weights from data

Borsetto Riccardo 

Gridelli Ivan 

## Introduction

The purpose of this notebook is to show how to learn weights of formulas from the observations given by a dataset.

The observations are provided by the dataset **Congressional Voting Records**, available at
https://www.kaggle.com/devvret/congressional-voting-records.
It includes votes for each of the U.S. House of Representatives Congressmen on the 16 key votes. 

Initially, we import the necessary libraries:

In [14]:
import sympy as sp
from math import log
import pandas as pd
from sympy.logic.inference import pl_true
from sklearn.impute import SimpleImputer

## Dataset

Then, we import the dataset:

In [15]:
pd.set_option('display.max_rows', 10)

df = pd.read_csv('./house-votes-84.csv')
df

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In this dataset, the rows list the individual representatives, while the columns list their votes for each of the sixteen areas.

In [16]:
dd = df[df['Class Name'] == 'democrat']
dr = df[df['Class Name'] == 'republican']
pd.DataFrame([[sum([1 for vote in dd[col] if vote == 'y']),
                         sum([1 for vote in dd[col] if vote == 'n']),
                         sum([1 for vote in dd[col] if vote == '?']), '***',
                         sum([1 for vote in dr[col] if vote == 'y']),
                         sum([1 for vote in dr[col] if vote == 'n']),
                         sum([1 for vote in dr[col] if vote == '?'])] for col in df.columns],
                         columns=['d_yes','d_no','d_?','***','r_yes','r_no','r_?']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
d_yes,0,156,120,231,14,55,123,200,218,188,124,129,36,73,90,160,173
d_no,0,102,119,29,245,200,135,59,45,60,139,126,213,179,167,91,12
d_?,0,9,28,7,8,12,9,8,4,19,4,12,18,15,10,16,82
***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***
r_yes,0,31,75,22,163,157,149,39,24,19,92,21,135,136,158,14,96
r_no,0,134,73,142,2,8,17,123,133,146,73,138,20,22,3,142,50
r_?,0,3,20,4,3,3,2,6,11,3,3,9,13,10,7,12,22


There are three type of votes: yes (y), no (n), unknown disposition (?). To assign a truth value to a propositional variable, we need to replace of the unknown dispositions, so then we can map **y** to **True** and **n** to **False**.

We implement two simple methods to replace **?**. 
The function **most_freq** replaces **?** with the most frequent value along the same column:

In [17]:
def most_freq(df):
    imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
    return pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

dff = most_freq(df)
dd = dff[dff['Class Name']=='democrat']
dr = dff[dff['Class Name']=='republican']
pd.DataFrame([[sum([1 for vote in dd[col] if vote=='y']),
                         sum([1 for vote in dd[col] if vote=='n']),
                         sum([1 for vote in dd[col] if vote=='?']), '***',
                         sum([1 for vote in dr[col] if vote=='y']),
                         sum([1 for vote in dr[col] if vote=='n']),
                         sum([1 for vote in dr[col] if vote=='?'])] for col in dff.columns],
                         columns=['d_yes','d_no','d_?','***','r_yes','r_no','r_?']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
d_yes,0,156,148,238,14,67,132,208,222,207,128,129,36,88,100,160,255
d_no,0,111,119,29,253,200,135,59,45,60,139,138,231,179,167,107,12
d_?,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***
r_yes,0,31,95,26,163,160,151,45,35,22,95,21,135,146,165,14,118
r_no,0,137,73,142,5,8,17,123,133,146,73,147,33,22,3,154,50
r_?,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
dff

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,n,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,y,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,y,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,y,y,y,y,n,y,y,y,n,y


The function **most_freq_by_party** replaces **?** with the most frequent value among representatives of the same class, along the same column:

In [39]:
def most_freq_by_party(df):
    df_democrats = df[df['Class Name']=='democrat']
    df_republicans = df[df['Class Name']=='republican']
    imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
    df_democrats_imputed = pd.DataFrame(imputer.fit_transform(df_democrats), columns = df.columns)
    df_republicans_imputed = pd.DataFrame(imputer.fit_transform(df_republicans), columns = df.columns)
    return pd.concat([df_democrats_imputed, df_republicans_imputed])



dfm = most_freq_by_party(df)


dd = dfm[dfm['Class Name']=='democrat']
dr = dfm[dfm['Class Name']=='republican']
pd.DataFrame([[sum([1 for vote in dd[col] if vote=='y']),
                         sum([1 for vote in dd[col] if vote == 'n']),
                         sum([1 for vote in dd[col] if vote == '?']), '***',
                         sum([1 for vote in dr[col] if vote == 'y']),
                         sum([1 for vote in dr[col] if vote == 'n']),
                         sum([1 for vote in dr[col] if vote == '?'])] for col in dfm.columns],
                         columns=['d_yes','d_no','d_?','***','r_yes','r_no','r_?']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
d_yes,0,165,148,238,14,55,123,208,222,207,124,141,36,73,90,176,255
d_no,0,102,119,29,253,212,144,59,45,60,143,126,231,194,177,91,12
d_?,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***,***
r_yes,0,31,95,22,166,160,151,39,24,19,95,21,148,146,165,14,118
r_no,0,137,73,146,2,8,17,129,144,149,73,147,20,22,3,154,50
r_?,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
dfm

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n
1,democrat,n,y,y,n,n,y,n,n,n,n,y,n,y,n,n,y
2,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y
3,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
4,democrat,n,y,n,y,y,y,n,n,n,n,n,n,n,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,republican,n,n,n,y,y,y,y,y,n,y,n,y,y,y,n,y
164,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
165,republican,n,y,n,y,y,y,n,n,n,n,y,y,y,y,n,y
166,republican,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,y


## Formulas

In order to define formulas, we need to define propositional variables.
We identify each voters political affiliation with its initial: d or r
and we identify each of the 16 votes with their position:

1. handicapped-infants,  
2. water-project-cost-sharing,
3. adoption-of-the-budget-resolution, 
4. physician-fee-freeze, 
5. el-salvador-aid, 
6. religious-groups-in-schools, 
7. anti-satellite-test-ban, 
8. aid-to-nicaraguan-contras, 
9. mx-missile, 
10. immigration, 
11. synfuels-corporation-cutback, 
12. education-spending, 
13. superfund-right-to-sue, 
14. crime, 
15. duty-free-exports, 
16. export-administration-act-south-africa 

In [20]:
d, r = sp.symbols('d r')
v = sp.symbols('v:' + str(len(df.columns)))
print(v)
# we also have a dummy variable v0 which will not be used

(v0, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16)


In order to compute the weights we have to define some functions 

The function **cnf** transforms a formula into an equivalent formula in conjunctive normal form.

In [21]:
def cnf(phi):
    phi = sp.to_cnf(phi, force=sp.true)
    phi = list(sp.And.make_args(phi))
    return [list(sp.Or.make_args(clause)) for clause in phi]

Next we have the function **cdp** and its auxiliary functions **restrict** and **unit_propagation** to count the models of a formula 

In [22]:
def restrict(phi, lit, truth):
    if truth == 0:
        lit = sp.Not(lit)
    return [[term for term in clause if term != sp.Not(lit)] for clause in phi if lit not in clause]

def unit_propagation(phi, n):
    i=0
    while i<len(phi):
        if len(phi[i])==1 and phi!=[] and phi[i]!=[]:
            lit = phi[i][0]
            phi = restrict(phi,lit,None)
            n = n+1
            i = 0
        else:
            i = i+1
    return [phi,n]

def cdp(phi, p, n):
    [phi, n] = unit_propagation(phi, n)
    if [] in phi:
        return 0
    if not phi:
        return 2**(p-n)
    lit = phi[0][0]
    return cdp(restrict(phi, lit, 0), p, n+1) + cdp(restrict(phi, lit, 1), p, n+1)

The function **true_obs** computes the number of observations in which a formula is satisfied, out of the total ones:

In [23]:
def true_obs(phi, df):
    n = 0
    for i in range(0, len(df)):
        if df.iloc[i][0] == 'democrat':
            model = {d:True, r:False}
        elif df.iloc[i][0] == 'republican':
            model = {d:False, r:True}
        model.update({v[x]:True if df.iloc[i][x] == 'y' else False for x in range(1, len(df.columns))})
        n = n + pl_true(phi, model, deep=True)
    return n

Finally, we can define the function **weight** that computes the weight of a formula given a dataset of observation, by combining the functions above.

The parameter mode allows for the choice of how to interpret the **?** votes: 
       mode = 1 uses **most_freq**
       mode = 2 uses **most_freq_by_party**
       
Enter the formula for which you would like to learn the weight of using the following syntax:
Symbols: & (and), | (or), >> (implies), ~ (not)
Variables: d, r, v1, v2, ..., v15, v16

In [42]:
def weight(phi, df, mode):  
    if phi is sp.S.true:
        return '+inf'
    if phi is (not sp.S.true):
        return '-inf'
    if mode == 1:
        df = most_freq(df)
    elif mode == 2:
        df = most_freq_by_party(df)
    p = len(phi.free_symbols)
    print('number of propositional variables', p)
    n = true_obs(phi, df)
    print('number of observations satisfying the formula', n)
    phi = cnf(phi)
    s = cdp(phi, p, 0)
    print('#SAT of the formula', s)
    if s == 0:
        return '-inf'
    d = len(df)
    print('total number of the observations', d)
    # we check if n is equal to d, which happens in two cases: the formula is valid or the formula is satisfied by all the observations
    if n == d:
        return '+inf'
    return log(n * (2 ** p - s) / ((d - n) * s))

Now we can list the formulas whose weight we would like to compute, for each vote1, vote2 and vote3:

* democrat → vote1
* republican → vote1
* democrat, vote1 → vote2
* republican, vote1 → vote2
* vote1, vote2 → democrat
* vote1, vote2 → republican
* vote1 → vote2
* vote1, vote2 → vote3

We compute the weights of the first two kind of formulas:

In [25]:
weights_1 = []
formulas_1 = []
weights_2 = []
formulas_2 = []

for i in range(1,len(df.columns)):
    formula_1 = d >> v[i]
    formulas_1.append(sp.pretty(formula_1))
    weights_1.append(weight(formula_1, df, 2))
    formula_2 = r >> v[i]
    formulas_2.append(sp.pretty(formula_1))
    weights_2.append(weight(formula_1, df, 2))

We print the result in a dataframe

In [26]:
df_weights = pd.DataFrame({'1': formulas_1, 'weights_1': weights_1, '2': formulas_2, 'weights_2': weights_2}) 

df_weights

,1,weights_1,2,weights_2
0,d → v₁,0.084557,d → v₁,0.084557
1,d → v₂,-0.121994,d → v₂,-0.121994
2,d → v₃,1.540445,d → v₃,1.540445
3,d → v₄,-1.427995,d → v₄,-1.427995
4,d → v₅,-1.048027,d → v₅,-1.048027
...,...,...,...,...
11,d → v₁₂,-1.222910,d → v₁₂,-1.222910
12,d → v₁₃,-0.881674,d → v₁₃,-0.881674
13,d → v₁₄,-0.721802,d → v₁₄,-0.721802
14,d → v₁₅,0.231170,d → v₁₅,0.231170


We can then do the same things for the other kind of formulas

In [27]:
weights_3 = []
formulas_3 = []
weights_4 = []
formulas_4 = []

for i in range(1, len(df.columns)):
    for j in range(1, len(df.columns)):
        formula_3 = (d & v[i]) >> v[j]
        formulas_3.append(sp.pretty(formula_3))
        weights_3.append(weight(formula_3, df, 2))
        formula_4 = (r & v[i]) >> v[j]
        formulas_4.append(sp.pretty(formula_4))
        weights_4.append(weight(formula_4, df, 2))

In [28]:
df_weights = pd.DataFrame({'3': formulas_3, 'weights_3': weights_3, '4': formulas_4, 'weights_4': weights_4}) 
df_weights

,3,weights_3,4,weights_4
0,(d ∧ v₁) → v₁,+inf,(r ∧ v₁) → v₁,+inf
1,(d ∧ v₁) → v₂,-0.328173,(r ∧ v₁) → v₂,1.705928
2,(d ∧ v₁) → v₃,1.457665,(r ∧ v₁) → v₃,0.986495
3,(d ∧ v₁) → v₄,-1.414186,(r ∧ v₁) → v₄,4.127134
4,(d ∧ v₁) → v₅,-1.179432,(r ∧ v₁) → v₅,3.023903
...,...,...,...,...
251,(d ∧ v₁₆) → v₁₂,-2.005698,(r ∧ v₁₆) → v₁₂,1.140336
252,(d ∧ v₁₆) → v₁₃,-1.691676,(r ∧ v₁₆) → v₁₃,1.035433
253,(d ∧ v₁₆) → v₁₄,-1.559566,(r ∧ v₁₆) → v₁₄,3.43168
254,(d ∧ v₁₆) → v₁₅,-0.50112,(r ∧ v₁₆) → v₁₅,-0.813291


In [29]:
weights_5 = []
formulas_5 = []
weights_6 = []
formulas_6 = []

for i in range(1,len(df.columns)):
    for j in range(1,len(df.columns)):
        formula_5 = (v[i] & v[j]) >> d
        formulas_5.append(sp.pretty(formula_5))
        weights_5.append(weight(formula_5, df, 2))
        formula_6 = (v[i] & v[j]) >> r
        formulas_6.append(sp.pretty(formula_6))
        weights_6.append(weight(formula_6, df, 2))

In [30]:
df_weights = pd.DataFrame({'5': formulas_5, 'weights_5': weights_5, '6': formulas_6, 'weights_6': weights_6}) 
df_weights

,5,weights_5,6,weights_6
0,v₁ → d,1.468815,v₁ → r,-0.606136
1,(v₁ ∧ v₂) → d,1.086636,(v₁ ∧ v₂) → r,-0.643699
2,(v₁ ∧ v₃) → d,1.911305,(v₁ ∧ v₃) → r,-1.314216
3,(v₁ ∧ v₄) → d,0.656780,(v₁ ∧ v₄) → r,2.733904
4,(v₁ ∧ v₅) → d,0.730699,(v₁ ∧ v₅) → r,0.769520
...,...,...,...,...
251,(v₁₂ ∧ v₁₆) → d,-0.723919,(v₁₂ ∧ v₁₆) → r,0.621518
252,(v₁₃ ∧ v₁₆) → d,-0.697575,(v₁₃ ∧ v₁₆) → r,-0.206794
253,(v₁₄ ∧ v₁₆) → d,-0.934309,(v₁₄ ∧ v₁₆) → r,-0.440427
254,(v₁₅ ∧ v₁₆) → d,1.616555,(v₁₅ ∧ v₁₆) → r,-1.521251


In [31]:
weights_7 = []
formulas_7 = []
for i in range(1, len(df.columns)):
    weights_7.append([])
    formulas_7.append([])

for i in range(1, len(df.columns)):
    for j in range(1, len(df.columns)):
        formula_7 = (v[i] >> v[j]) 
        formulas_7[i-1].append(sp.pretty(formula_7))
        weights_7[i-1].append(weight(formula_7, df, 2))

In [32]:
df_temp_7 = []
for i in range(len(formulas_7)):       
    df_temp_7.append(formulas_7[i])
    df_temp_7.append(weights_7[i])

df_temp_7 = list(map(list, zip(*df_temp_7)))

In [33]:
df_weights_7 = pd.DataFrame(df_temp_7)
df_weights_7

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,True,+inf,v₂ → v₁,-0.245835,v₃ → v₁,0.110348,v₄ → v₁,-0.415792,v₅ → v₁,-0.557015,...,v₁₂ → v₁,-0.446567,v₁₃ → v₁,-0.53719,v₁₄ → v₁,-0.759775,v₁₅ → v₁,0.391479,v₁₆ → v₁,-0.835028
1,v₁ → v₂,0.346178,True,+inf,v₃ → v₂,-0.14501,v₄ → v₂,0.502572,v₅ → v₂,0.331357,...,v₁₂ → v₂,0.361137,v₁₃ → v₂,0.486201,v₁₄ → v₂,-0.075223,v₁₅ → v₂,0.189975,v₁₆ → v₂,-0.625619
2,v₁ → v₃,1.30683,v₂ → v₃,0.059115,True,+inf,v₄ → v₃,-0.456758,v₅ → v₃,-0.527237,...,v₁₂ → v₃,-0.426084,v₁₃ → v₃,-0.415792,v₁₄ → v₃,-0.615889,v₁₅ → v₃,1.833793,v₁₆ → v₃,-0.110395
3,v₁ → v₄,-0.576737,v₂ → v₄,-0.310824,v₃ → v₄,-1.213682,True,+inf,v₅ → v₄,1.085906,...,v₁₂ → v₄,1.504077,v₁₃ → v₄,0.677147,v₁₄ → v₄,0.406871,v₁₅ → v₄,-0.683559,v₁₆ → v₄,-1.380932
4,v₁ → v₅,-0.363792,v₂ → v₅,-0.039459,v₃ → v₅,-0.955842,v₄ → v₅,2.758602,True,+inf,...,v₁₂ → v₅,1.882731,v₁₃ → v₅,1.247738,v₁₄ → v₅,0.898179,v₁₅ → v₅,-0.547116,v₁₆ → v₅,-1.10321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,v₁ → v₁₂,-0.566889,v₂ → v₁₂,-0.363792,v₃ → v₁₂,-1.149198,v₄ → v₁₂,1.657006,v₅ → v₁₂,0.898179,...,True,+inf,v₁₃ → v₁₂,0.677147,v₁₄ → v₁₂,0.391479,v₁₅ → v₁₂,-0.645015,v₁₆ → v₁₂,-1.334178
12,v₁ → v₁₃,-0.300105,v₂ → v₁₃,0.136503,v₃ → v₁₃,-0.816292,v₄ → v₁₃,1.741927,v₅ → v₁₃,1.368989,...,v₁₂ → v₁₃,1.577996,True,+inf,v₁₄ → v₁₃,0.640503,v₁₅ → v₁₃,-0.477046,v₁₆ → v₁₃,-1.038824
13,v₁ → v₁₄,-0.15643,v₂ → v₁₄,0.071794,v₃ → v₁₄,-0.664327,v₄ → v₁₄,3.581201,v₅ → v₁₄,2.463853,...,v₁₂ → v₁₄,2.758602,v₁₃ → v₁₄,1.540445,True,+inf,v₁₅ → v₁₄,-0.332134,v₁₆ → v₁₄,-0.73132
14,v₁ → v₁₅,0.302112,v₂ → v₁₅,-0.426084,v₃ → v₁₅,0.21733,v₄ → v₁₅,-0.58656,v₅ → v₁₅,-0.788095,...,v₁₂ → v₁₅,-0.58656,v₁₃ → v₁₅,-0.759775,v₁₄ → v₁₅,-0.965081,True,+inf,v₁₆ → v₁₅,-0.835028


In [34]:
formulas_8 = []
weights_8 = []
for i in range(1, len(df.columns)):
    weights_8.append([])
    formulas_8.append([])

for i in range(1, len(df.columns)):
    for j in range(1, len(df.columns)):
        for k in range(1,len(df.columns)):
            formula_8 = (v[i] & v[j]) >> v[k]
            formulas_8[i-1].append(sp.pretty(formula_8))
            weights_8[i-1].append(weight(formula_8, df, 2))

In [35]:
df_temp_8 = []
for i in range(len(formulas_8)):       
    df_temp_8.append(formulas_8[i])
    df_temp_8.append(weights_8[i])

df_temp_8 = list(map(list, zip(*df_temp_8)))

In [36]:
df_weights_8 = pd.DataFrame(df_temp_8)
df_weights_8

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,True,+inf,(v₁ ∧ v₂) → v₁,+inf,(v₁ ∧ v₃) → v₁,+inf,(v₁ ∧ v₄) → v₁,+inf,(v₁ ∧ v₅) → v₁,+inf,...,(v₁ ∧ v₁₂) → v₁,+inf,(v₁ ∧ v₁₃) → v₁,+inf,(v₁ ∧ v₁₄) → v₁,+inf,(v₁ ∧ v₁₅) → v₁,+inf,(v₁ ∧ v₁₆) → v₁,+inf
1,v₁ → v₂,0.346178,(v₁ ∧ v₂) → v₂,+inf,(v₁ ∧ v₃) → v₂,-0.328173,(v₁ ∧ v₄) → v₂,1.803594,(v₁ ∧ v₅) → v₂,1.196804,...,(v₁ ∧ v₁₂) → v₂,1.457665,(v₁ ∧ v₁₃) → v₂,1.457665,(v₁ ∧ v₁₄) → v₂,0.621518,(v₁ ∧ v₁₅) → v₂,0.050881,(v₁ ∧ v₁₆) → v₂,-0.440427
2,v₁ → v₃,1.30683,(v₁ ∧ v₂) → v₃,0.851371,(v₁ ∧ v₃) → v₃,+inf,(v₁ ∧ v₄) → v₃,0.939619,(v₁ ∧ v₅) → v₃,0.730699,...,(v₁ ∧ v₁₂) → v₃,1.086636,(v₁ ∧ v₁₃) → v₃,0.894629,(v₁ ∧ v₁₄) → v₃,0.587291,(v₁ ∧ v₁₅) → v₃,2.031432,(v₁ ∧ v₁₆) → v₃,0.809708
3,v₁ → v₄,-0.576737,(v₁ ∧ v₂) → v₄,-0.588108,(v₁ ∧ v₃) → v₄,-1.293865,(v₁ ∧ v₄) → v₄,+inf,(v₁ ∧ v₅) → v₄,0.894629,...,(v₁ ∧ v₁₂) → v₄,1.457665,(v₁ ∧ v₁₃) → v₄,0.521691,(v₁ ∧ v₁₄) → v₄,0.316869,(v₁ ∧ v₁₅) → v₄,-0.813291,(v₁ ∧ v₁₆) → v₄,-1.384488
4,v₁ → v₅,-0.363792,(v₁ ∧ v₂) → v₅,-0.393321,(v₁ ∧ v₃) → v₅,-1.125832,(v₁ ∧ v₄) → v₅,3.023903,(v₁ ∧ v₅) → v₅,+inf,...,(v₁ ∧ v₁₂) → v₅,1.705928,(v₁ ∧ v₁₃) → v₅,1.086636,(v₁ ∧ v₁₄) → v₅,0.851371,(v₁ ∧ v₁₅) → v₅,-0.710795,(v₁ ∧ v₁₆) → v₅,-1.200577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,(v₁ ∧ v₁₆) → v₁₂,-1.364554,(v₁₆ ∧ v₂) → v₁₂,-1.147402,(v₁₆ ∧ v₃) → v₁₂,-1.959703,(v₁₆ ∧ v₄) → v₁₂,0.939619,(v₁₆ ∧ v₅) → v₁₂,0.213574,...,(v₁₂ ∧ v₁₆) → v₁₂,+inf,(v₁₃ ∧ v₁₆) → v₁₂,-0.054067,(v₁₄ ∧ v₁₆) → v₁₂,-0.361097,(v₁₅ ∧ v₁₆) → v₁₂,-1.472917,v₁₆ → v₁₂,-1.334178
252,(v₁ ∧ v₁₆) → v₁₃,-1.125832,(v₁₆ ∧ v₂) → v₁₃,-0.670841,(v₁₆ ∧ v₃) → v₁₃,-1.654204,(v₁₆ ∧ v₄) → v₁₃,1.035433,(v₁₆ ∧ v₅) → v₁₃,0.65678,...,(v₁₂ ∧ v₁₆) → v₁₃,0.851371,(v₁₃ ∧ v₁₆) → v₁₃,+inf,(v₁₄ ∧ v₁₆) → v₁₃,-0.132528,(v₁₅ ∧ v₁₆) → v₁₃,-1.304056,v₁₆ → v₁₃,-1.038824
253,(v₁ ∧ v₁₆) → v₁₄,-0.980829,(v₁₆ ∧ v₂) → v₁₄,-0.74989,(v₁₆ ∧ v₃) → v₁₄,-1.511625,(v₁₆ ∧ v₄) → v₁₄,3.023903,(v₁₆ ∧ v₅) → v₁₄,1.705928,...,(v₁₂ ∧ v₁₆) → v₁₄,1.911305,(v₁₃ ∧ v₁₆) → v₁₄,0.730699,(v₁₄ ∧ v₁₆) → v₁₄,+inf,(v₁₅ ∧ v₁₆) → v₁₄,-1.168798,v₁₆ → v₁₄,-0.73132
254,(v₁ ∧ v₁₆) → v₁₅,-0.409183,(v₁₆ ∧ v₂) → v₁₅,-0.886757,(v₁₆ ∧ v₃) → v₁₅,-0.559616,(v₁₆ ∧ v₄) → v₁₅,-0.862565,(v₁₆ ∧ v₅) → v₁₅,-1.071093,...,(v₁₂ ∧ v₁₆) → v₁₅,-0.898746,(v₁₃ ∧ v₁₆) → v₁₅,-1.071093,(v₁₄ ∧ v₁₆) → v₁₅,-1.28364,(v₁₅ ∧ v₁₆) → v₁₅,+inf,v₁₆ → v₁₅,-0.835028


## Example

In [47]:
psi = ~(v[1] >> v[2])


weight(psi, df, 1)

number of propositional variables 2
number of observations satisfying the formula 82
#SAT of the formula 1
total number of the observations 435


-0.361136521000934